# <center>Introduction to Curricular Simulations</center>

<center>
    <b>Gregory L. Heileman$^\dagger$, Jiacheng Zhang$^\ddagger$ and Hayden W. Free$^\bullet$</b> <br>
    $^\dagger$Department of Electrical & Computer Engineering <br>
    University of Arizona <br>
    heileman@arizona.edu <br>
    $^\ddagger$Department of Computer Science <br>
    jiachengzhang1@arizona.edu <br>
    University of Arizona<br>
    $^\bullet$Department of Computer Science <br>
    hayden.free@uky.edu <br>
    University of Kentucky
</center>

## 1. Introduction

This notebook demonstrates how to use the simulation capabilites that are included as a part of the [CurricularAnalytics toolbox](https://github.com/CurricularAnalytics/CurricularAnalytics.jl). If you would like to become more familiar with the notions behind curricular analytics, we suggest you read <cite data-cite="he:18">Heileman, et. al, (2018)</cite>, and also examine the Introduction to the Curricular Analytics Toolbox notebook that accompanies this notebook.

The simulation capabilites include the ability to simulate the flow of students through a curriculum, towards graduation, using discrete event simulation. Specifically, a population of students attempt to complete the selected curriculum, by taking courses in the order prescribed by the curriculum. At each step (semester) of the simulation a given student enrolls in a set of courses, earning either a passing of failing grade in each.  At the end of a given semester, if a student has passed all of the courses in the curriculum, they are deemed a graduate.  If a student has not yet gradauted, then they may stop out (according to a prescribed stop-out model), or enroll in the next set of courses available to them. One of the intended uses of these simulation capabilities is to estimate the impact that particular curricular changes or instructional improvements will have on student progress. 

The simulation framework (shown below) was orginally developed by <cite data-cite="hi:18">Hickman, (2014)</cite>, and  subsequent development has allowed it to be integrated into the CurricularAnalytics toolbox. 

<img src="SimulationFramework.png" width="600">

Notice that the simulation "engine" requires three inputs: a curriculum, a model for students, and a model for student peformance. The results of the simualtion are returned in an object that may be viewed using the `simulation_report()` function, as demonstrated below. 

In order to perform curricular simulations, first load the Curricular Analytics toolbox modules:

In [14]:
using CurricularAnalytics, CurricularVisualization

## 2. Setting up the Simulation Environment

The first thing we will do is read in a degree plan. The student in the simulation will attempt to complete the curriculum associated with the degree plan in the order that prescribed in the degree plan.  Specifically, in each semester each student will enroll in courses they have not yet taken in the order specified by the degree plan until they reach the maximum allowed number of credit hours.

### 2.1 Reading the Degree Plan

The following commands read in a degree plan stored in the CSV file format, and then display a visualization of the resulting plan.

In [45]:
AE_degree_plan = read_csv("Univ_of_Arizona-Aero.csv")
visualize(AE_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Aerospace Engineering\",\"dp_name\":\"2019-20 Degree Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 125\",\"metrics\":{\"centrality\":0,\"complexity\":38.0,\"blocking factor\":29,\"delay factor\":9.0},\"nameSub\":\"Calculus I w/ Applications\",\"id\":1,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 151\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Gen Chemistry I\",\"id\":2,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 101\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"First-year Comp.\",\"id\":3,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":4,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGR 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Intro to Engineering\",\"id\":4,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier I General Ed.\",\"id\":5,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":6,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 129\",\"metrics\":{\"centrality\":387,\"complexity\":31.0,\"blocking factor\":22,\"delay factor\":9.0},\"nameSub\":\"Calculus II\",\"id\":6,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"AME 105\",\"metrics\":{\"centrality\":28,\"complexity\":16.0,\"blocking factor\":9,\"delay factor\":7.0},\"nameSub\":\"Intro. to MATLAB I\",\"id\":7,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"PHYS 141\",\"metrics\":{\"centrality\":277,\"complexity\":28.0,\"blocking factor\":19,\"delay factor\":9.0},\"nameSub\":\"Intro Mechanics\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"First-year Comp.\",\"id\":9,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":10,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ECE 175\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Programming for Eng. Apps.\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier I General Ed. 1\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":8,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":6,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CE 214\",\"metrics\":{\"centrality\":264,\"complexity\":24.0,\"blocking factor\":15,\"delay factor\":9.0},\"nameSub\":\"Statics\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH  223\",\"metrics\":{\"centrality\":252,\"complexity\":25

In [16]:
basic_metrics(AE_degree_plan)
AE_degree_plan.metrics

Dict{String,Any} with 8 entries:
  "total credit hours"         => 129
  "avg. credits per term"      => 16.125
  "min. credits in a term"     => 15
  "term credit hour std. dev." => 0.927025
  "number of terms"            => 8
  "max. credits in a term"     => 18
  "min. credit term"           => 4
  "max. credit term"           => 3

In [17]:
CS_degree_plan = read_csv("Univ_of_Arizona-CS.csv")
visualize(CS_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Computer Science\",\"dp_name\":\"2019-20 Degree Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":1,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CSC 110\",\"metrics\":{\"centrality\":25,\"complexity\":12.0,\"blocking factor\":7,\"delay factor\":5.0},\"nameSub\":\"Intro to Computer Programming I\",\"id\":1,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MATH 120R\",\"metrics\":{\"centrality\":0,\"complexity\":15.0,\"blocking factor\":10,\"delay factor\":5.0},\"nameSub\":\"Pre-Calculus\",\"id\":2,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 101\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"English Composition I\",\"id\":3,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"1st Semester Second Language\",\"id\":4,\"credits\":4,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":5,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 120\",\"metrics\":{\"centrality\":25,\"complexity\":11.0,\"blocking factor\":6,\"delay factor\":5.0},\"nameSub\":\"Intro to Computer Programming II\",\"id\":5,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":6,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 122A/B\",\"metrics\":{\"centrality\":3,\"complexity\":4.0,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Calculus I\",\"id\":6,\"credits\":5,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"English Composition II\",\"id\":7,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"2nd Semester Second Language\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":5,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 210\",\"metrics\":{\"centrality\":20,\"complexity\":9.0,\"blocking factor\":4,\"delay factor\":5.0},\"nameSub\":\"Software Development\",\"id\":9,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":5,\"target_id\":10,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 245\",\"metrics\":{\"centrality\":5,\"complexity\":6.0,\"blocking factor\":1,\"delay factor\":5.0},\"nameSub\":\"Intro to Discrete Structures\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":11,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 129\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"Calculus II\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier 1 General Education 1\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay facto

In [18]:
EE_degree_plan = read_csv("Ga_Tech-EE.csv")
visualize(EE_degree_plan, notebook=true, scale=1.0)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Electrical Engineering\",\"dp_name\":\"2017-18 Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":9.0,\"blocking factor\":4,\"delay factor\":5.0},\"nameSub\":\"CHEM 1310\",\"id\":1,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":23.0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"MATH 1551\",\"id\":2,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":16.0,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"MATH 1553\",\"id\":3,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":17.0,\"blocking factor\":10,\"delay factor\":7.0},\"nameSub\":\"CS 1371\",\"id\":4,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Wellness Elective\",\"id\":5,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"ENGL 1101\",\"id\":6,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":29,\"complexity\":17.0,\"blocking factor\":9,\"delay factor\":8.0},\"nameSub\":\"PHYS 2211\",\"id\":7,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":34,\"complexity\":17.0,\"blocking factor\":10,\"delay factor\":7.0},\"nameSub\":\"MATH 1552\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":11,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"ECE 2020\",\"id\":9,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"History/Government Elective\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":11,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"ENGL 1102\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":7,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":29,\"complexity\":16.0,\"blocking factor\":8,\"delay factor\":8.0},\"nameSub\":\"PHYS 2212\",\"id\":12,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":8,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"MATH 2552\",\"id\":13,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":14,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":3,\"target_id\":14,\"type\":\"CurriculumPrerequisite\"},{\"sou

In [19]:
basic_metrics(EE_degree_plan)
EE_degree_plan.metrics

Dict{String,Any} with 8 entries:
  "total credit hours"         => 134
  "avg. credits per term"      => 16.75
  "min. credits in a term"     => 15
  "term credit hour std. dev." => 1.08972
  "number of terms"            => 8
  "max. credits in a term"     => 18
  "min. credit term"           => 4
  "max. credit term"           => 5

### 2.2 Creating the Student Cohort

The following command will create an inital cohort of students `n` using a simple enrollment model. Specifically, with this simple model, all students are assumed equally likely (or unlikely) to pass a given class according the course pass rate probability model.

In [20]:
enrollment_model = Enrollment  # use the Enrollment module to determine if/when student may enroll in a course
stopouts = true  # assume that student may stop out of the cohort
n = 1000   # student cohort size will be 100
students = simple_students(n);  # create a student cohort

### 2.3 The Course Performance Model

First let's see what happens if the instructional complexity is the same for both programs. Specifically, we will use course pass rates as a proxy for instructional complexity, and we'll set the pass rates for all courses at 90%. 

In [21]:
performance_model = PassRate
real_passrate = false
set_passrates(AE_degree_plan.curriculum.courses, 0.9)
set_passrates(CS_degree_plan.curriculum.courses, 0.9)
course_passrate = 0.9;

### 2.4 Setting the Simulation Parameters

In [33]:
max_credits = 18  # the maximum number of credit hours a student may enroll in during a semester
duration_lock = false # rather than simulating until no students are left in the cohort, run for a fixed number of terms
num_terms = 12  # the maximum number of terms in the simulation
course_attempt_limit = 2;  # number of times a student may attempt a course

### 2.5 Running the Simulation 
The `simulation` function is used to execute the simulation.  Depending upon how many students are in the cohort, this may take some time to run.

In [34]:
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);

In order to view the results of the simulation, use the `simulation_report` function:

In [35]:
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 2
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 268
Graduation Rate: 26.8%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015, 0.159, 0.26, 0.268, 0.0]
Average time to degree: 9.380597014925373 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 732
Number of Students Reaching Max Attempts: 531
Stop-out Rate: 73.2%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.091, 0.342, 0.514, 0.614, 0.672, 0.699, 0.707, 0.717, 0.723, 0.731, 0.732, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.091, 0.181, 0.172, 0.184, 0.195, 0.2, 0.202, 0.202, 0.201,

│ 38  │ Stability and Control of Aero. Vehicles │ 0.0%  │ 0.0%  │ 0.0%  │
│ 39  │ Orbital Mechanicans and Space Flt.      │ 0.0%  │ 0.0%  │ 0.0%  │
│ 40  │ Senior Colloquium                       │ 0.0%  │ 54.9% │ 68.6% │
│ 41  │ Aerospace Eng.Design                    │ 0.0%  │ 0.0%  │ 0.0%  │
│ 42  │ Num. Meth. in Fluid Mech. & Heat Xfer   │ 0.0%  │ 0.0%  │ 0.0%  │
│ 43  │ Technical Elective 1                    │ 0.0%  │ 0.0%  │ 0.0%  │
│ 44  │ Technical Elective 2                    │ 0.0%  │ 0.0%  │ 0.0%  │
│ 45  │ Tier II General Ed. 2                   │ 0.0%  │ 0.0%  │ 0.0%  │
│ 46  │ GRAD RATE                               │ 0.0%  │ 0.0%  │ 0.0%  │

│ Row │ TERM4 │ TERM5 │ TERM6 │ TERM7 │ TERM8 │ TERM9 │ TERM10 │ TERM11 │
├─────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┤
│ 1   │ 94.7% │ 94.7% │ 94.7% │ 94.7% │ 94.7% │ 94.7% │ 94.7%  │ 94.7%  │
│ 2   │ 91.5% │ 91.5% │ 91.5% │ 91.5% │ 91.5% │ 91.5% │ 91.5%  │ 91.5%  │
│ 3   │ 98.3% │ 98.3% │ 98.3% │ 98.3%

Now let's run the same set of students through the Computer Science curriculum:

In [36]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 2
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 388
Graduation Rate: 38.800000000000004%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.187, 0.364, 0.388, 0.0, 0.0]
Average time to degree: 8.579896907216495 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 612
Number of Students Reaching Max Attempts: 344
Stop-out Rate: 61.2%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.088, 0.274, 0.333, 0.442, 0.508, 0.534, 0.569, 0.592, 0.61, 0.612, 0.0, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.088, 0.184, 0.212, 0.242, 0.259, 0.263, 0.268, 0.268,

│ 13  │ 69.3% │ 69.3% │ 69.3% │ 69.3% │ 69.3% │ 69.3%  │ 69.3%  │ 69.3%  │
│ 14  │ 57.8% │ 60.1% │ 60.3% │ 60.3% │ 60.3% │ 60.3%  │ 60.3%  │ 60.3%  │
│ 15  │ 59.3% │ 61.4% │ 61.4% │ 61.4% │ 61.4% │ 61.4%  │ 61.4%  │ 61.4%  │
│ 16  │ 62.2% │ 62.4% │ 62.4% │ 62.4% │ 62.4% │ 62.4%  │ 62.4%  │ 62.4%  │
│ 17  │ 59.0% │ 60.9% │ 60.9% │ 60.9% │ 60.9% │ 60.9%  │ 60.9%  │ 60.9%  │
│ 18  │ 54.0% │ 56.5% │ 56.6% │ 56.6% │ 56.6% │ 56.6%  │ 56.6%  │ 56.6%  │
│ 19  │ 48.0% │ 53.5% │ 53.7% │ 53.7% │ 53.7% │ 53.7%  │ 53.7%  │ 53.7%  │
│ 20  │ 42.4% │ 51.8% │ 52.1% │ 52.1% │ 52.1% │ 52.1%  │ 52.1%  │ 52.1%  │
│ 21  │ 32.6% │ 49.6% │ 51.4% │ 51.4% │ 51.4% │ 51.4%  │ 51.4%  │ 51.4%  │
│ 22  │ 27.4% │ 49.4% │ 51.3% │ 51.3% │ 51.3% │ 51.3%  │ 51.3%  │ 51.3%  │
│ 23  │ 8.3%  │ 44.0% │ 48.5% │ 48.5% │ 48.5% │ 48.5%  │ 48.5%  │ 48.5%  │
│ 24  │ 0.2%  │ 41.1% │ 47.8% │ 48.5% │ 48.5% │ 48.5%  │ 48.5%  │ 48.5%  │
│ 25  │ 0.2%  │ 36.9% │ 47.7% │ 48.2% │ 48.2% │ 48.2%  │ 48.2%  │ 48.2%  │
│ 26  │ 0.0%  │ 30.7% │ 4

In [37]:
real_passrate = false
set_passrates(EE_degree_plan.curriculum.courses, 0.9)
simulation = simulate(EE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Electrical Engineering, BS -- 2017-18 Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 2
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 424
Graduation Rate: 42.4%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06, 0.297, 0.417, 0.424, 0.0]
Average time to degree: 9.174528301886792 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 576
Number of Students Reaching Max Attempts: 288
Stop-out Rate: 57.6%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.079, 0.257, 0.328, 0.402, 0.461, 0.5, 0.527, 0.543, 0.567, 0.576, 0.576, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.079, 0.209, 0.234, 0.269, 0.28, 0.285, 0.288, 0.288, 0.288, 0.288,

Next we will use a model to determine whether or not a student passes a course. We will use actual pass/fail rates computed using historical data for the courses in the degree plan shown above. 

In [38]:
course_passrate = 0.9  # use if a course is not contained in the CSV file
real_passrate = true  # use the actual pass rates, rather than course_passrate for all courses
set_passrates_from_csv(AE_degree_plan.curriculum.courses, "./Student_Grades_sp17_to_fall19.csv", course_passrate)

Note: A more realistic model for predicting student performance could be used here. Specifically, a more realistic model might:

- take student demographics into account, including the major they are in,
- take prior grades into account when predicting future grades,
- take into account factors that influence student stopout, e.g., academic standing, GPA, unment need, etc.

Learning the model pararmeters using actual student data would improve the fidelity of the simulation.

In [39]:
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 2
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 288
Graduation Rate: 28.799999999999997%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021, 0.184, 0.279, 0.288, 0.0]
Average time to degree: 9.319444444444445 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 712
Number of Students Reaching Max Attempts: 466
Stop-out Rate: 71.2%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.08, 0.333, 0.491, 0.599, 0.65, 0.671, 0.681, 0.689, 0.703, 0.711, 0.712, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.08, 0.192, 0.21, 0.228, 0.244, 0.247, 0.246, 

│ 8   │ 75.4% │ 75.4% │ 75.4% │ 75.4% │ 75.4% │ 75.4% │ 75.4%  │ 75.4%  │
│ 9   │ 83.8% │ 83.8% │ 83.8% │ 83.8% │ 83.8% │ 83.8% │ 83.8%  │ 83.8%  │
│ 10  │ 79.6% │ 79.6% │ 79.6% │ 79.6% │ 79.6% │ 79.6% │ 79.6%  │ 79.6%  │
│ 11  │ 74.7% │ 74.7% │ 74.7% │ 74.7% │ 74.7% │ 74.7% │ 74.7%  │ 74.7%  │
│ 12  │ 47.9% │ 51.7% │ 51.8% │ 51.8% │ 51.8% │ 51.8% │ 51.8%  │ 51.8%  │
│ 13  │ 49.3% │ 53.1% │ 53.3% │ 53.3% │ 53.3% │ 53.3% │ 53.3%  │ 53.3%  │
│ 14  │ 46.0% │ 49.9% │ 50.2% │ 50.2% │ 50.2% │ 50.2% │ 50.2%  │ 50.2%  │
│ 15  │ 60.4% │ 61.2% │ 61.2% │ 61.2% │ 61.2% │ 61.2% │ 61.2%  │ 61.2%  │
│ 16  │ 63.0% │ 63.6% │ 63.6% │ 63.6% │ 63.6% │ 63.6% │ 63.6%  │ 63.6%  │
│ 17  │ 57.6% │ 60.6% │ 60.6% │ 60.6% │ 60.6% │ 60.6% │ 60.6%  │ 60.6%  │
│ 18  │ 46.3% │ 50.0% │ 50.0% │ 50.0% │ 50.0% │ 50.0% │ 50.0%  │ 50.0%  │
│ 19  │ 18.8% │ 37.8% │ 42.6% │ 42.7% │ 42.7% │ 42.7% │ 42.7%  │ 42.7%  │
│ 20  │ 15.0% │ 34.3% │ 39.1% │ 39.8% │ 39.8% │ 39.8% │ 39.8%  │ 39.8%  │
│ 21  │ 8.2%  │ 30.7% │ 38.1% │ 38.5% 

Now, let's see what happens if we use the actual course pass rates in the computer science program.

In [40]:
set_passrates_from_csv(CS_degree_plan.curriculum.courses, "./Student_Grades_sp17_to_fall19.csv", course_passrate)
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 2
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 269
Graduation Rate: 26.900000000000002%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.098, 0.241, 0.269, 0.0, 0.0]
Average time to degree: 8.739776951672862 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 731
Number of Students Reaching Max Attempts: 536
Stop-out Rate: 73.1%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.068, 0.36, 0.441, 0.535, 0.624, 0.671, 0.7, 0.719, 0.729, 0.731, 0.0, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.068, 0.144, 0.16, 0.18, 0.19, 0.193, 0.195, 0.195, 0.19

## 3. What-if Analyses

These simulation capabilities allow us to conduct what-if analyses around the impact that changes to curricular structure or instructional improvements will have on student success.  First, let's consider how changing the number of allowed attempts (from 2 to 3) would impact graduation rates in both of these programs.

In [41]:
course_attempt_limit = 3
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 588
Graduation Rate: 58.8%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.011, 0.251, 0.498, 0.579, 0.588]
Average time to degree: 9.72278911564626 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 410
Number of Students Reaching Max Attempts: 116
Stop-out Rate: 41.0%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.074, 0.188, 0.243, 0.324, 0.368, 0.385, 0.398, 0.4, 0.403, 0.406, 0.408, 0.41]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.074, 0.188, 0.225, 0.262, 0.285, 0.288, 0.294, 0.294, 0.2

│ 11  │ 85.5%  │ 86.4%  │ 86.4%  │ 86.4%  │ 86.4%  │ 86.4%  │ 86.4%  │ 86.4%  │
│ 12  │ 58.4%  │ 70.2%  │ 72.6%  │ 73.3%  │ 73.4%  │ 73.4%  │ 73.4%  │ 73.4%  │
│ 13  │ 60.3%  │ 71.2%  │ 73.8%  │ 74.2%  │ 74.2%  │ 74.2%  │ 74.2%  │ 74.2%  │
│ 14  │ 53.8%  │ 68.8%  │ 71.4%  │ 72.0%  │ 72.0%  │ 72.0%  │ 72.0%  │ 72.0%  │
│ 15  │ 72.6%  │ 78.3%  │ 78.7%  │ 78.7%  │ 78.7%  │ 78.7%  │ 78.7%  │ 78.7%  │
│ 16  │ 74.2%  │ 78.5%  │ 78.9%  │ 78.9%  │ 78.9%  │ 78.9%  │ 78.9%  │ 78.9%  │
│ 17  │ 69.6%  │ 76.4%  │ 77.5%  │ 77.5%  │ 77.5%  │ 77.5%  │ 77.5%  │ 77.5%  │
│ 18  │ 56.2%  │ 69.7%  │ 72.3%  │ 72.6%  │ 72.6%  │ 72.6%  │ 72.6%  │ 72.6%  │
│ 19  │ 21.4%  │ 49.8%  │ 62.3%  │ 66.6%  │ 67.4%  │ 67.6%  │ 67.6%  │ 67.6%  │
│ 20  │ 17.2%  │ 44.8%  │ 59.2%  │ 64.5%  │ 65.5%  │ 65.7%  │ 65.7%  │ 65.7%  │
│ 21  │ 8.0%   │ 42.0%  │ 58.9%  │ 63.9%  │ 64.8%  │ 64.9%  │ 64.9%  │ 64.9%  │
│ 22  │ 35.9%  │ 62.1%  │ 70.5%  │ 71.8%  │ 71.9%  │ 71.9%  │ 71.9%  │ 71.9%  │
│ 23  │ 0.0%   │ 7.5%   │ 37.9%  │ 55.9%

In [42]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 544
Graduation Rate: 54.400000000000006%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.142, 0.427, 0.526, 0.544, 0.0]
Average time to degree: 8.987132352941176 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 456
Number of Students Reaching Max Attempts: 147
Stop-out Rate: 45.6%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.077, 0.2, 0.296, 0.355, 0.393, 0.419, 0.436, 0.443, 0.447, 0.453, 0.456, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.077, 0.2, 0.236, 0.281, 0.298, 0.305, 0.309, 0.309

Next, notice that CSC 110 -- Intro to Computer Programming I is clearly a gateway course for the computer science program at the University of Arizona, and that is has a relatively low success rate. There are a few other courses with the CSC prefix that also have low success rates. What would happen if the instruction and instructional support were changed in a way that enabled the students taking CSC 110, CSC 120 and CSC 353 to obtain a 90% pass rate? 

In [43]:
convert_ids(CS_degree_plan.curriculum)
csc110 = course(CS_degree_plan.curriculum, "CSC", "110", "Intro to Computer Programming I", "")
csc120 = course(CS_degree_plan.curriculum, "CSC", "120", "Intro to Computer Programming II", "")
csc210 = course(CS_degree_plan.curriculum, "CSC", "210", "Software Development", "")
csc345 = course(CS_degree_plan.curriculum, "CSC", "345", "Analysis of Discrete Structures", "")
csc352 = course(CS_degree_plan.curriculum, "CSC", "352", "Systems Programming & Unix", "")

csc110.passrate = csc120.passrate = csc210.passrate = csc345.passrate = csc352.passrate = 0.9;

In [44]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 638
Graduation Rate: 63.800000000000004%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.216, 0.55, 0.626, 0.638, 0.0]
Average time to degree: 8.818181818181818 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 362
Number of Students Reaching Max Attempts: 68
Stop-out Rate: 36.2%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.069, 0.189, 0.241, 0.286, 0.322, 0.338, 0.351, 0.352, 0.358, 0.361, 0.362, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.069, 0.189, 0.219, 0.259, 0.28, 0.289, 0.294, 0.29

# References

Heileman, G. L., Abdallah, C.T., Slim, A., and Hickman, M. (2018). Curricular analytics: A framework for quantifying the impact of curricular reforms and pedagogical innovations. www.arXiv.org, arXiv:1811.09676 [cs.CY].

Hickman, M. (2014). Development of a Curriculum Analysis and Simulation Library with Applications in Curricular Analytics. MS thesis, University of New Mexico,
Albuquerque, NM.